# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, desc
from pyspark.sql.functions import sum as ssum

spark = (
    SparkSession
    .builder
    .appName("Wrangling Data")
    .getOrCreate()
)

In [2]:
sdf = spark.read.json("data/sparkify_log_small.json")

# Question 1

Which page did user id "" (empty string) NOT visit?

In [3]:
sdf.where("userId = ''").select("page").distinct().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [5]:
# TODO: use this space to explore the behavior of the user with an empty string
sdf.where("userId = ''").groupby("page").count().toPandas()

,page,count
0,Home,187
1,About,15
2,Login,126
3,Help,8


# Question 3

How many female users do we have in the data set?

In [4]:
# TODO: write your code to answer question 3
sdf.select("userId", "gender").distinct().groupby("gender").count().toPandas()

,gender,count
0,F,462
1,None,1
2,M,501


# Question 4

How many songs were played from the most played artist?

In [14]:
artist = (
    sdf
    .dropna(subset="artist")
    .groupby("artist")
    .count()
    .sort("count", ascending=False)
    .limit(1)
    .select("artist")
).collect()[0][0]
print("Top artist:", artist)

Top artist: Coldplay


In [24]:
sdf.where(f"artist = '{artist}'").select("song").count()

83

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [27]:
# TODO: write your code to answer question 5
is_home = udf(lambda x: int(x == "Home"), IntegerType())

In [33]:
window = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

In [47]:
sdf_agg = (
    sdf
    .filter("page IN ('NextSong', 'Home')")
    .withColumn("homeVisit", is_home("page"))
    .withColumn("period", ssum("homeVisit").over(window))
    .select(
        "userId",
        "song",
        "page",
        "homeVisit",
        "period"
    )
)

In [ ]:
(
    sdf_agg
    .filter("page = 'NextSong'")
    .groupBy("userId", "period")
    .agg({"period": "count"})
    .agg({"count(period)": "avg"})
).toPandas()